In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler


from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict, extract_features_2
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load clean gdp data, keep only year, region and real_gdp columns
poland = pd.read_csv("data/tabular_data_poland.csv")
log_bin_columns = ["allangle_snow_free_hq" + "_log_" + str(i) for i in range(1, 11)]

# get the data for 2021, 2022 and before 2022
poland_2022 = poland[poland["year"] == 2022]
poland = poland[poland["year"].astype(int) < 2022]
poland_2021 = poland[poland["year"] == 2021]
poland_2021.reset_index(drop=True, inplace=True)
poland_2022.reset_index(drop=True, inplace=True)
poland.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(poland), 609, 911, 1))
y = np.zeros(len(poland))

# load the images
for i in range(len(poland)):

    # get year, region, and gdp
    year = poland["year"][i]
    region = poland["region"][i]
    gdp_value = poland["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(poland["year"] == 2021)[0]
train_indices = np.where(poland["year"] != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(poland_2022),  609, 911, 1))
for i in range(len(poland_2022)):

    year = poland_2022["year"][i]
    region = poland_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [3]:
poland = pd.read_csv("data/tabular_data_poland.csv")
poland = poland[poland["year"] < 2023]
poland_sum = poland[['year', 'region', 'allangle_snow_free_hq_sum'] + log_bin_columns]

model = define_cnn(n_features = 4, n_conv = 2, n_dense = 4, input_shape = (609, 911, 1), res_x = 305, res_y = 455)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

poland_stage_2, poland_pred = extract_features_2(model, poland, 2022, X, X_pred, 4)
selected_columns =  ["feature_" + str(i) for i in range(1, 5)] + log_bin_columns + ['allangle_snow_free_hq_sum']

full_data = pd.concat([poland_stage_2, poland_pred])
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data = pd.merge(full_data, poland_sum, on=['year', 'region'])
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)
full_data_diff = full_data_diff[full_data_diff["year"] != 2012]

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
pred_data_diff = full_data_diff[full_data_diff["year"] == 2022]
pred_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)




Epoch 1/50


2/2 [==============================] - 9s 3s/step - loss: 0.9754 - mae: 0.6574 - val_loss: 0.0877 - val_mae: 0.2686
Epoch 2/50
2/2 [==============================] - 2s 859ms/step - loss: 0.1376 - mae: 0.2749 - val_loss: 0.2210 - val_mae: 0.4390
Epoch 3/50
2/2 [==============================] - 2s 962ms/step - loss: 0.0786 - mae: 0.2119 - val_loss: 0.3361 - val_mae: 0.5458
Epoch 4/50
2/2 [==============================] - 1s 830ms/step - loss: 0.0642 - mae: 0.1845 - val_loss: 0.4591 - val_mae: 0.6270
Epoch 5/50
2/2 [==============================] - 2s 988ms/step - loss: 0.0505 - mae: 0.1574 - val_loss: 0.5627 - val_mae: 0.6745
Epoch 6/50
2/2 [==============================] - 2s 1s/step - loss: 0.0394 - mae: 0.1409 - val_loss: 0.6516 - val_mae: 0.6999
Epoch 7/50
2/2 [==============================] - 2s 780ms/step - loss: 0.0310 - mae: 0.1237 - val_loss: 0.7176 - val_mae: 0.7163
Epoch 8/50
2/2 [==============================] - 1s 653ms/step - loss: 0.0266 - mae: 0.1152

In [4]:
# train a neural network
train_data_nn = train_data_diff.copy()
pred_data_nn = pred_data_diff.copy()

train_data_nn = pd.get_dummies(train_data_nn, columns=["region"])
pred_data_nn = pd.get_dummies(pred_data_nn, columns=["region"])

X_train_nn = train_data_nn.drop(columns=["year", "real_gdp"])
X_train_nn = X_train_nn.drop(columns=log_bin_columns)
# X_train_nn = X_train_nn.drop(columns=["allangle_snow_free_hq_mean"])
y_train_nn = train_data_nn["real_gdp"]

X_test_nn = pred_data_nn.drop(columns=["year", "real_gdp"])
X_test_nn = X_test_nn.drop(columns=log_bin_columns)
# X_test_nn = X_test_nn.drop(columns=["allangle_snow_free_hq_mean"])
y_test_nn = pred_data_nn["real_gdp"]

X_train_nn = np.array(X_train_nn, dtype=np.float32)
X_test_nn = np.array(X_test_nn, dtype=np.float32)
y_train_nn = np.array(y_train_nn, dtype=np.float32)
y_test_nn = np.array(y_test_nn, dtype=np.float32)

In [5]:
# scale the data
scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train_nn)
X_test_nn = scaler.transform(X_test_nn)

# fit the model on the X_train and y_train
model = Sequential()
# model.add(Dense(256, activation="relu", input_dim=X_train_nn.shape[1]))  # First layer
# model.add(Dense(128, activation="relu"))  # First layer
# model.add(Dense(64, activation="relu"))  # First layer
model.add(Dense(32, activation="relu", input_dim=X_train_nn.shape[1]))  # Fourth layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(8, activation="relu"))  # New additional layer
model.add(Dense(1, activation = 'linear')) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])


model.fit(X_train_nn, y_train_nn, epochs=500, batch_size=64, validation_split=0.2)

# # get the predictions
# boolean_columns = [col for col in X_test_nn.columns if X_test_nn[col].dtype == bool]
# for col in boolean_columns:
#     X_test_nn[col] = X_test_nn[col].astype(int)

y_pred = model.predict(X_test_nn).flatten()


# # get the predictions
# boolean_columns = [col for col in X_test_nn.columns if X_test_nn[col].dtype == bool]
# for col in boolean_columns:
#     X_test_nn[col] = X_test_nn[col].astype(int)

y_hat = model.predict(X_test_nn).flatten()


Epoch 1/500
2/2 [==============================] - 1s 220ms/step - loss: 60758412.0000 - mae: 5033.5737 - val_loss: 25964048.0000 - val_mae: 3766.4683
Epoch 2/500
2/2 [==============================] - 0s 41ms/step - loss: 60757660.0000 - mae: 5033.5225 - val_loss: 25963662.0000 - val_mae: 3766.4358
Epoch 3/500
2/2 [==============================] - 0s 50ms/step - loss: 60756940.0000 - mae: 5033.4717 - val_loss: 25963262.0000 - val_mae: 3766.4036
Epoch 4/500
2/2 [==============================] - 0s 46ms/step - loss: 60756192.0000 - mae: 5033.4219 - val_loss: 25962854.0000 - val_mae: 3766.3706
Epoch 5/500
2/2 [==============================] - 0s 48ms/step - loss: 60755444.0000 - mae: 5033.3730 - val_loss: 25962446.0000 - val_mae: 3766.3367
Epoch 6/500
2/2 [==============================] - 0s 43ms/step - loss: 60754764.0000 - mae: 5033.3252 - val_loss: 25962032.0000 - val_mae: 3766.3020
Epoch 7/500
2/2 [==============================] - 0s 39ms/step - loss: 60753992.0000 - mae: 5033.

In [6]:
y_hat

array([11324.204  ,  5330.855  ,  7167.6265 ,  7393.0444 ,  3149.7878 ,
        8194.65   , 23398.646  ,  1748.6278 ,    82.85541,  2298.103  ,
        7988.1724 , 12099.55   ,  1951.3784 ,   305.72742, 12493.538  ,
        2007.7446 ], dtype=float32)

In [7]:
pred_data_diff["gdp_pred"] = y_hat
poland_pred = pred_data_diff[["year", "region", "gdp_pred", "real_gdp"]]
poland_pred.to_csv("poland_predictions_new_2.csv", index=False)